In [1]:
from google.colab import drive
drive.mount('/content/drive')

directory = '/content/drive/MyDrive/Colab Notebooks/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd


# one-step ahead forecasting

In [3]:
# Function to convert time series data into supervised learning data
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [4]:
ts_data = pd.read_csv(directory + "ts_data1.csv")
ts_data.head()

,Date,Births
0,1959-01-01,35
1,1959-01-02,32
2,1959-01-03,30
3,1959-01-04,31
4,1959-01-05,44


In [5]:
# Extract the 'Births' column for modeling
data = ts_data['Births'].values
data = data.astype('float32')

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data.reshape(-1, 1))

# Split into train and test sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [6]:
# Reshape into X=t and Y=t+1
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [7]:
print("trainX.shape", trainX.shape)
print("testX.shape", testX.shape)
print("trainY.shape", trainY.shape)
print("testY.shape", testY.shape)

print("trainX[0]:\n", trainX[0])
print("\n\n trainY[0]:\n", np.round(trainY[0],1))

trainX.shape (234, 10)
testX.shape (111, 10)
trainY.shape (234,)
testY.shape (111,)
trainX[0]:
 [0.24000001 0.18       0.13999999 0.16000003 0.42000002 0.12
 0.44       0.39999998 0.3        0.07999998]


 trainY[0]:
 0.3


In [8]:
# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [9]:
# Create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=0)

In [10]:
# Making predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Inverting predictions back to original scale
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# Model summary and prediction outputs
model.summary(), trainPredict[:5], testPredict[:5]  # Displaying first 5 predictions for brevity

4/4 [==============================] - 0s 2ms/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4)                 96        
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 101 (404.00 Byte)
Trainable params: 101 (404.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


(None,
 array([[39.69404 ],
        [40.507206],
        [40.164627],
        [41.84803 ],
        [41.327293]], dtype=float32),
 array([[41.277622],
        [42.14753 ],
        [41.276615],
        [40.6391  ],
        [41.130062]], dtype=float32))

# 3-step ahead forecasting

In [12]:
def create_dataset_v2(dataset, look_back=1, look_ahead=3):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - look_ahead + 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back:i + look_back + look_ahead, 0])
    return np.array(dataX), np.array(dataY)

In [13]:
# Reshape into X=t and Y=t+look_ahead
look_back = 10
look_ahead = 3
trainX, trainY = create_dataset_v2(train, look_back, look_ahead)
testX, testY = create_dataset_v2(test, look_back, look_ahead)

In [14]:
print("trainX.shape", trainX.shape)
print("testX.shape", testX.shape)
print("trainY.shape", trainY.shape)
print("testY.shape", testY.shape)

print("trainX[0]:\n", trainX[0])
print("\n\n trainY[0]:\n", np.round(trainY[0],1))

trainX.shape (232, 10)
testX.shape (109, 10)
trainY.shape (232, 3)
testY.shape (109, 3)
trainX[0]:
 [0.24000001 0.18       0.13999999 0.16000003 0.42000002 0.12
 0.44       0.39999998 0.3        0.07999998]


 trainY[0]:
 [0.3 0.2 0.6]


In [15]:
# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [16]:
# Create and fit the LSTM network
model2 = Sequential()
model2.add(LSTM(4, input_shape=(look_back, 1)))
model2.add(Dense(look_ahead))
model2.compile(loss='mean_squared_error', optimizer='adam')
model2.fit(trainX, trainY, epochs=50, batch_size=1, verbose=0)

In [24]:
test_instance = testX[-1]
print("\n\n test_instance:", test_instance)
test_label = testY[-1:]
print("\n\n test_label:", test_label)

# Reshape test_instance for prediction
test_instance_reshaped = test_instance.reshape((1, test_instance.shape[0], test_instance.shape[1]))

testPredict = model2.predict(test_instance_reshaped, verbose=0)
print("\n\n testPredict:", testPredict)


# Inverting predictions back to original scale
testPredict = scaler.inverse_transform(testPredict)
test_label = scaler.inverse_transform(test_label)

print("\n\n test_label:", np.round(test_label,1))
print("\n\n testPredict:", np.round(testPredict,1))



 test_instance: [[0.38      ]
 [0.38      ]
 [0.59999996]
 [0.32      ]
 [0.33999997]
 [0.3       ]
 [0.42000002]
 [0.22000003]
 [0.28000003]
 [0.58      ]]


 test_label: [[0.5        0.64000005 0.54      ]]


 testPredict: [[0.34364888 0.33742204 0.3452357 ]]


 test_label: [[48. 55. 50.]]


 testPredict: [[40.2 39.9 40.3]]
